<a href="https://colab.research.google.com/github/Yahyawi99/Yahyawi99/blob/main/PFE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
!pip install --upgrade pip
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install unsloth
!pip install transformers datasets accelerate peft bitsandbytes


Looking in indexes: https://download.pytorch.org/whl/cu118
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 67.1 MB/s eta 0:00:00


In [ ]:
from unsloth import FastLanguageModel
import torch

# Use a 4-bit quantized LLaMA 2 7B
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-2-7b-bnb-4bit",
    max_seq_length = 2048,
    dtype = torch.float16,
    load_in_4bit = True,
)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


RuntimeError: Failed to import transformers.integrations.integration_utils because of the following error (look up to see its traceback):
Failed to import transformers.modeling_utils because of the following error (look up to see its traceback):
Detected that PyTorch and torchvision were compiled with different CUDA major versions. PyTorch has CUDA Version=11.8 and torchvision has CUDA Version=12.4. Please reinstall the torchvision that matches your PyTorch install.

In [32]:
# Clean old versions first
!pip uninstall -y torch torchvision torchaudio
!pip install --upgrade pip

# Install Unsloth and required packages
!pip install unsloth
!pip install transformers datasets accelerate peft bitsandbytes



Found existing installation: torch 2.6.0
Uninstalling torch-2.6.0:
  Successfully uninstalled torch-2.6.0
Found existing installation: torchvision 0.21.0
Uninstalling torchvision-0.21.0:
  Successfully uninstalled torchvision-0.21.0
Found existing installation: torchaudio 2.6.0+cu118
Uninstalling torchaudio-2.6.0+cu118:
  Successfully uninstalled torchaudio-2.6.0+cu118
  Using cached torch-2.6.0-cp311-cp311-manylinux1_x86_64.whl.metadata (28 kB)
  Using cached torchvision-0.21.0-cp311-cp311-manylinux1_x86_64.whl.metadata (6.1 kB)
Using cached torch-2.6.0-cp311-cp311-manylinux1_x86_64.whl (766.7 MB)
Using cached torchvision-0.21.0-cp311-cp311-manylinux1_x86_64.whl (7.2 MB)


In [1]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! Llama 3 is up to 8k
dtype = None
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
    "unsloth/gemma-7b-it-bnb-4bit",
    "unsloth/gemma-2b-bnb-4bit",
    "unsloth/gemma-2b-it-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",
]

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit", # Llama-3 70b also works (just change the model name)
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch Gemma3ForConditionalGeneration.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.51.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [2]:


model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2025.3.19 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [41]:
from datasets import Dataset

# Hardcoded dataset: just input/output pairs
schema = """# Table Schemas and Relationships

-- employees
employee_id INT PRIMARY KEY,
first_name VARCHAR(20),
last_name VARCHAR(25),
email VARCHAR(25),
phone_number VARCHAR(20),
hire_date DATE,
job_id VARCHAR(10),
salary DECIMAL(8,2),
commission_pct DECIMAL(2,2),
manager_id INT,
department_id INT

-- departments
department_id INT PRIMARY KEY,
department_name VARCHAR(30),
manager_id INT,
location_id INT

-- jobs
job_id VARCHAR(10) PRIMARY KEY,
job_title VARCHAR(35),
min_salary DECIMAL(8,2),
max_salary DECIMAL(8,2)

-- locations
location_id INT PRIMARY KEY,
street_address VARCHAR(40),
postal_code VARCHAR(12),
city VARCHAR(30),
state_province VARCHAR(25),
country_id CHAR(2)

-- countries
country_id CHAR(2) PRIMARY KEY,
country_name VARCHAR(40),
region_id INT

-- regions
region_id INT PRIMARY KEY,
region_name VARCHAR(25)

-- job_grades
grade_level CHAR(1) PRIMARY KEY,
lowest_sal DECIMAL(8,2),
highest_sal DECIMAL(8,2)

# Relationships:
employees.department_id → departments.department_id
employees.manager_id → employees.employee_id
employees.job_id → jobs.job_id
departments.manager_id → employees.employee_id
departments.location_id → locations.location_id
locations.country_id → countries.country_id
countries.region_id → regions.region_id
employees.salary → within job_grades.lowest_sal–highest_sal (logical)
"""


data = [
  {
    "instruction": "Display employees who do not have a commission.",
    "input": "employees (employee_id, first_name, last_name, commission_pct, salary, department_id, job_id, hire_date, manager_id, email)",
    "output": "SELECT first_name, last_name FROM employees WHERE commission_pct IS NULL;"
  },
  {
    "instruction": "List all employees and their job titles.",
    "input": "employees (employee_id, first_name, last_name, job_id), jobs (job_id, job_title)",
    "output": "SELECT e.first_name, e.last_name, j.job_title FROM employees e JOIN jobs j ON e.job_id = j.job_id;"
  },
  {
    "instruction": "Find employees who have been hired after a certain date.",
    "input": "employees (employee_id, first_name, last_name, hire_date)",
    "output": "SELECT first_name, last_name FROM employees WHERE hire_date > TO_DATE('2021-01-01', 'YYYY-MM-DD');"
  },
  {
    "instruction": "Find the average salary for each department.",
    "input": "employees (employee_id, first_name, last_name, salary, department_id), departments (department_id, department_name)",
    "output": "SELECT d.department_name, AVG(e.salary) FROM employees e JOIN departments d ON e.department_id = d.department_id GROUP BY d.department_name;"
  },
  {
    "instruction": "List employees who earn more than the average salary in their department.",
    "input": "employees (employee_id, first_name, last_name, salary, department_id), departments (department_id, department_name)",
    "output": "SELECT e.first_name, e.last_name, e.salary FROM employees e WHERE e.salary > (SELECT AVG(salary) FROM employees WHERE department_id = e.department_id);"
  },
  {
    "instruction": "Display employees with no manager.",
    "input": "employees (employee_id, first_name, last_name, manager_id)",
    "output": "SELECT first_name, last_name FROM employees WHERE manager_id IS NULL;"
  },
  {
    "instruction": "Find the total salary expense for each department.",
    "input": "employees (employee_id, first_name, last_name, salary, department_id), departments (department_id, department_name)",
    "output": "SELECT d.department_name, SUM(e.salary) FROM employees e JOIN departments d ON e.department_id = d.department_id GROUP BY d.department_name;"
  },
  {
    "instruction": "List employees with the same job title and salary.",
    "input": "employees (employee_id, first_name, last_name, job_id, salary), jobs (job_id, job_title)",
    "output": "SELECT e1.first_name, e1.last_name FROM employees e1 JOIN employees e2 ON e1.salary = e2.salary AND e1.job_id = e2.job_id WHERE e2.job_id = 'IT_PROG';"
  },
  {
    "instruction": "Find employees with a commission percentage greater than 20%.",
    "input": "employees (employee_id, first_name, last_name, commission_pct)",
    "output": "SELECT first_name, last_name FROM employees WHERE commission_pct > 0.2;"
  },
  {
    "instruction": "List employees who report to a specific manager.",
    "input": "employees (employee_id, first_name, last_name, manager_id)",
    "output": "SELECT first_name, last_name FROM employees WHERE manager_id = 102;"
  },
  {
    "instruction": "List the department names and the number of employees in each department.",
    "input": "employees (employee_id, first_name, last_name, department_id), departments (department_id, department_name)",
    "output": "SELECT d.department_name, COUNT(e.employee_id) FROM employees e JOIN departments d ON e.department_id = d.department_id GROUP BY d.department_name;"
  },
  {
    "instruction": "Find the highest paid employee in each department.",
    "input": "employees (employee_id, first_name, last_name, salary, department_id), departments (department_id, department_name)",
    "output": "SELECT d.department_name, e.first_name, e.last_name, e.salary FROM employees e JOIN departments d ON e.department_id = d.department_id WHERE e.salary = (SELECT MAX(salary) FROM employees WHERE department_id = e.department_id);"
  },
  {
    "instruction": "Display employees with a hire date after January 1, 2020.",
    "input": "employees (employee_id, first_name, last_name, hire_date)",
    "output": "SELECT first_name, last_name FROM employees WHERE hire_date > TO_DATE('2020-01-01', 'YYYY-MM-DD');"
  },
  {
    "instruction": "List the number of employees for each job title.",
    "input": "employees (employee_id, first_name, last_name, job_id), jobs (job_id, job_title)",
    "output": "SELECT j.job_title, COUNT(e.employee_id) FROM employees e JOIN jobs j ON e.job_id = j.job_id GROUP BY j.job_title;"
  },
  {
    "instruction": "List all employees along with their manager's name.",
    "input": "employees (employee_id, first_name, last_name, manager_id), managers (employee_id, first_name, last_name)",
    "output": "SELECT e.first_name, e.last_name, m.first_name AS manager_first_name, m.last_name AS manager_last_name FROM employees e LEFT JOIN employees m ON e.manager_id = m.employee_id;"
  },
  {
    "instruction": "Find employees who have not taken any vacation.",
    "input": "employees (employee_id, first_name, last_name), vacation (employee_id, vacation_start, vacation_end)",
    "output": "SELECT e.first_name, e.last_name FROM employees e LEFT JOIN vacation v ON e.employee_id = v.employee_id WHERE v.employee_id IS NULL;"
  },
  {
    "instruction": "List employees who work in a specific location.",
    "input": "employees (employee_id, first_name, last_name, location_id), locations (location_id, city)",
    "output": "SELECT e.first_name, e.last_name FROM employees e JOIN locations l ON e.location_id = l.location_id WHERE l.city = 'London';"
  },
  {
    "instruction": "Find employees who have a salary greater than the average salary of all employees.",
    "input": "employees (employee_id, first_name, last_name, salary)",
    "output": "SELECT first_name, last_name, salary FROM employees WHERE salary > (SELECT AVG(salary) FROM employees);"
  },
  {
    "instruction": "List employees whose job title is 'Manager'.",
    "input": "employees (employee_id, first_name, last_name, job_id), jobs (job_id, job_title)",
    "output": "SELECT e.first_name, e.last_name FROM employees e JOIN jobs j ON e.job_id = j.job_id WHERE j.job_title = 'Manager';"
  },
   {
    "instruction": "Find the top 5 highest paid employees in each department.",
    "input": "employees (employee_id, first_name, last_name, salary, department_id), departments (department_id, department_name)",
    "output": "SELECT e.first_name, e.last_name, e.salary, d.department_name FROM (SELECT e.first_name, e.last_name, e.salary, e.department_id, RANK() OVER (PARTITION BY e.department_id ORDER BY e.salary DESC) AS salary_rank FROM employees e) e JOIN departments d ON e.department_id = d.department_id WHERE e.salary_rank <= 5;"
  },
  {
    "instruction": "Find the employee with the highest salary in each department.",
    "input": "employees (employee_id, first_name, last_name, salary, department_id), departments (department_id, department_name)",
    "output": "SELECT e.first_name, e.last_name, e.salary, d.department_name FROM employees e JOIN departments d ON e.department_id = d.department_id WHERE (e.salary, e.department_id) IN (SELECT MAX(salary), department_id FROM employees GROUP BY department_id);"
  },
  {
    "instruction": "List the number of employees working under each manager, including managers who have no employees.",
    "input": "employees (employee_id, first_name, last_name, manager_id), managers (employee_id, first_name, last_name)",
    "output": "SELECT m.first_name AS manager_first_name, m.last_name AS manager_last_name, COUNT(e.employee_id) AS employee_count FROM employees e RIGHT JOIN employees m ON e.manager_id = m.employee_id GROUP BY m.first_name, m.last_name;"
  },
  {
    "instruction": "Find the average salary of employees in each job category within a department.",
    "input": "employees (employee_id, first_name, last_name, salary, job_id, department_id), jobs (job_id, job_title), departments (department_id, department_name)",
    "output": "SELECT d.department_name, j.job_title, AVG(e.salary) AS avg_salary FROM employees e JOIN jobs j ON e.job_id = j.job_id JOIN departments d ON e.department_id = d.department_id GROUP BY d.department_name, j.job_title;"
  },
  {
    "instruction": "Identify employees who have a salary greater than the average salary of their respective job titles in their department.",
    "input": "employees (employee_id, first_name, last_name, salary, job_id, department_id), jobs (job_id, job_title)",
    "output": "SELECT e.first_name, e.last_name, e.salary, e.job_id, j.job_title FROM employees e JOIN jobs j ON e.job_id = j.job_id WHERE e.salary > (SELECT AVG(salary) FROM employees WHERE job_id = e.job_id AND department_id = e.department_id);"
  },
  {
    "instruction": "Find employees who are in the same department and have the same salary but different job titles.",
    "input": "employees (employee_id, first_name, last_name, salary, department_id, job_id), jobs (job_id, job_title)",
    "output": "SELECT e1.first_name, e1.last_name, e1.salary, e1.job_id AS job_id_1, j1.job_title AS job_title_1, e2.first_name, e2.last_name, e2.job_id AS job_id_2, j2.job_title AS job_title_2 FROM employees e1 JOIN employees e2 ON e1.salary = e2.salary AND e1.department_id = e2.department_id AND e1.job_id != e2.job_id JOIN jobs j1 ON e1.job_id = j1.job_id JOIN jobs j2 ON e2.job_id = j2.job_id;"
  },
  {
    "instruction": "Find the total salary expense for each job title within each department, excluding employees with a salary lower than 3000.",
    "input": "employees (employee_id, first_name, last_name, salary, job_id, department_id), jobs (job_id, job_title), departments (department_id, department_name)",
    "output": "SELECT d.department_name, j.job_title, SUM(e.salary) AS total_salary_expense FROM employees e JOIN jobs j ON e.job_id = j.job_id JOIN departments d ON e.department_id = d.department_id WHERE e.salary >= 3000 GROUP BY d.department_name, j.job_title;"
  },
  {
    "instruction": "List employees whose salary is higher than the average salary of employees with the same job title but in different departments.",
    "input": "employees (employee_id, first_name, last_name, salary, job_id, department_id), jobs (job_id, job_title), departments (department_id, department_name)",
    "output": "SELECT e.first_name, e.last_name, e.salary, j.job_title FROM employees e JOIN jobs j ON e.job_id = j.job_id WHERE e.salary > (SELECT AVG(salary) FROM employees WHERE job_id = e.job_id AND department_id != e.department_id);"
  },
  {
    "instruction": "Find the number of employees who have been with the company for more than 5 years in each department.",
    "input": "employees (employee_id, first_name, last_name, hire_date, department_id), departments (department_id, department_name)",
    "output": "SELECT d.department_name, COUNT(e.employee_id) AS employee_count FROM employees e JOIN departments d ON e.department_id = d.department_id WHERE e.hire_date <= ADD_MONTHS(SYSDATE, -12 * 5) GROUP BY d.department_name;"
  },
  {
    "instruction": "Find the employees who have the same job title but different hire dates, sorted by hire date.",
    "input": "employees (employee_id, first_name, last_name, hire_date, job_id), jobs (job_id, job_title)",
    "output": "SELECT e1.first_name, e1.last_name, e1.hire_date, e1.job_id, j.job_title FROM employees e1 JOIN employees e2 ON e1.job_id = e2.job_id AND e1.hire_date != e2.hire_date JOIN jobs j ON e1.job_id = j.job_id ORDER BY e1.hire_date;"
  },
  {
    "instruction": "Find the department with the highest total salary expense, and list all employees in that department.",
    "input": "employees (employee_id, first_name, last_name, salary, department_id), departments (department_id, department_name)",
    "output": "SELECT e.first_name, e.last_name, e.salary, d.department_name FROM employees e JOIN departments d ON e.department_id = d.department_id WHERE e.department_id = (SELECT department_id FROM employees GROUP BY department_id ORDER BY SUM(salary) DESC FETCH FIRST 1 ROWS ONLY);"
  },
  {
    "instruction": "List the top 3 employees with the highest salaries in each job category, including their job title.",
    "input": "employees (employee_id, first_name, last_name, salary, job_id), jobs (job_id, job_title)",
    "output": "SELECT e.first_name, e.last_name, e.salary, j.job_title FROM (SELECT e.first_name, e.last_name, e.salary, e.job_id, RANK() OVER (PARTITION BY e.job_id ORDER BY e.salary DESC) AS salary_rank FROM employees e) e JOIN jobs j ON e.job_id = j.job_id WHERE e.salary_rank <= 3;"
  },
  {
    "instruction": "Find all employees who report to a manager with a specific title, e.g., 'Sales Manager'.",
    "input": "employees (employee_id, first_name, last_name, manager_id, job_id), jobs (job_id, job_title)",
    "output": "SELECT e.first_name, e.last_name FROM employees e JOIN jobs j ON e.job_id = j.job_id WHERE e.manager_id IN (SELECT employee_id FROM employees WHERE job_id = (SELECT job_id FROM jobs WHERE job_title = 'Sales Manager'));"
  },
  {
    "instruction": "Find the total salary expense for each job title in each department, grouped by department name and job title.",
    "input": "employees (employee_id, first_name, last_name, salary, job_id, department_id), jobs (job_id, job_title), departments (department_id, department_name)",
    "output": "SELECT d.department_name, j.job_title, SUM(e.salary) AS total_salary_expense FROM employees e JOIN jobs j ON e.job_id = j.job_id JOIN departments d ON e.department_id = d.department_id GROUP BY d.department_name, j.job_title;"
  },
  {
    "instruction": "Identify employees who have been with the company the longest in each department.",
    "input": "employees (employee_id, first_name, last_name, hire_date, department_id), departments (department_id, department_name)",
    "output": "SELECT e.first_name, e.last_name, e.hire_date, d.department_name FROM employees e JOIN departments d ON e.department_id = d.department_id WHERE e.hire_date = (SELECT MIN(hire_date) FROM employees WHERE department_id = e.department_id);"
  },
  {
    "instruction": "Find employees who have had a salary increase within the last year.",
    "input": "employees (employee_id, first_name, last_name, salary, hire_date)",
    "output": "SELECT first_name, last_name, salary FROM employees WHERE hire_date > ADD_MONTHS(SYSDATE, -12);"
  },
  {
    "instruction": "Find the departments with the most employees and list the department names and number of employees.",
    "input": "employees (employee_id, first_name, last_name, department_id), departments (department_id, department_name)",
    "output": "SELECT d.department_name, COUNT(e.employee_id) AS employee_count FROM employees e JOIN departments d ON e.department_id = d.department_id GROUP BY d.department_name ORDER BY employee_count DESC FETCH FIRST 1 ROWS ONLY;"
  },
  {
    "instruction": "Identify employees who have the highest salary in their department and return their names, salary, and department name.",
    "input": "employees (employee_id, first_name, last_name, salary, department_id), departments (department_id, department_name)",
    "output": "SELECT e.first_name, e.last_name, e.salary, d.department_name FROM employees e JOIN departments d ON e.department_id = d.department_id WHERE e.salary = (SELECT MAX(salary) FROM employees WHERE department_id = e.department_id);"
  },
  {
    "instruction": "List all job titles and their average salaries across all departments.",
    "input": "employees (employee_id, salary, job_id, department_id), jobs (job_id, job_title)",
    "output": "SELECT j.job_title, AVG(e.salary) AS avg_salary FROM employees e JOIN jobs j ON e.job_id = j.job_id GROUP BY j.job_title;"
  },
  {
    "instruction": "Find the total salary for all employees in a specific department, and display the department name and the total salary.",
    "input": "employees (employee_id, salary, department_id), departments (department_id, department_name)",
    "output": "SELECT d.department_name, SUM(e.salary) AS total_salary FROM employees e JOIN departments d ON e.department_id = d.department_id GROUP BY d.department_name;"
  },
  {
    "instruction": "Find the employees whose salary is above the average salary in their respective job titles and department.",
    "input": "employees (employee_id, first_name, last_name, salary, job_id, department_id), jobs (job_id, job_title)",
    "output": "SELECT e.first_name, e.last_name, e.salary, j.job_title FROM employees e JOIN jobs j ON e.job_id = j.job_id WHERE e.salary > (SELECT AVG(salary) FROM employees WHERE job_id = e.job_id AND department_id = e.department_id);"
  },
  {
    "instruction": "List all employees who joined in the year 2020 and their department names.",
    "input": "employees (employee_id, first_name, last_name, hire_date, department_id), departments (department_id, department_name)",
    "output": "SELECT e.first_name, e.last_name, d.department_name FROM employees e JOIN departments d ON e.department_id = d.department_id WHERE EXTRACT(YEAR FROM e.hire_date) = 2020;"
  },
  {
    "instruction": "Find the job title with the highest average salary across all departments.",
    "input": "employees (employee_id, salary, job_id), jobs (job_id, job_title)",
    "output": "SELECT j.job_title, AVG(e.salary) AS avg_salary FROM employees e JOIN jobs j ON e.job_id = j.job_id GROUP BY j.job_title ORDER BY avg_salary DESC FETCH FIRST 1 ROWS ONLY;"
  },
  {
    "instruction": "List all employees who report to a manager with a salary greater than 5000.",
    "input": "employees (employee_id, first_name, last_name, manager_id, salary), jobs (job_id, job_title)",
    "output": "SELECT e.first_name, e.last_name, e.salary FROM employees e JOIN employees m ON e.manager_id = m.employee_id WHERE m.salary > 5000;"
  },
  {
    "instruction": "List all employees who are in the same department and have the same job title, but different salaries.",
    "input": "employees (employee_id, first_name, last_name, salary, job_id, department_id), jobs (job_id, job_title)",
    "output": "SELECT e1.first_name, e1.last_name, e1.salary, e1.job_id, j.job_title FROM employees e1 JOIN employees e2 ON e1.department_id = e2.department_id AND e1.job_id = e2.job_id AND e1.salary != e2.salary JOIN jobs j ON e1.job_id = j.job_id;"
  },
  {
    "instruction": "Identify employees who have a salary greater than 3000 and have been employed for more than 3 years.",
    "input": "employees (employee_id, first_name, last_name, salary, hire_date)",
    "output": "SELECT first_name, last_name, salary FROM employees WHERE salary > 3000 AND hire_date <= ADD_MONTHS(SYSDATE, -36);"
  },
  {
    "instruction": "List the departments with the highest total salary expense and display the department name and total salary.",
    "input": "employees (employee_id, salary, department_id), departments (department_id, department_name)",
    "output": "SELECT d.department_name, SUM(e.salary) AS total_salary FROM employees e JOIN departments d ON e.department_id = d.department_id GROUP BY d.department_name ORDER BY total_salary DESC FETCH FIRST 1 ROWS ONLY;"
  },
  {
    "instruction": "Find the department that has the most employees with a salary greater than 4000.",
    "input": "employees (employee_id, salary, department_id), departments (department_id, department_name)",
    "output": "SELECT d.department_name, COUNT(e.employee_id) AS employee_count FROM employees e JOIN departments d ON e.department_id = d.department_id WHERE e.salary > 4000 GROUP BY d.department_name ORDER BY employee_count DESC FETCH FIRST 1 ROWS ONLY;"
  },
  {
    "instruction": "Find the total number of employees with a salary greater than 5000 for each job title.",
    "input": "employees (employee_id, salary, job_id), jobs (job_id, job_title)",
    "output": "SELECT j.job_title, COUNT(e.employee_id) AS employee_count FROM employees e JOIN jobs j ON e.job_id = j.job_id WHERE e.salary > 5000 GROUP BY j.job_title;"
  },
  {
    "instruction": "List all employees who are earning a salary above the average salary in their department.",
    "input": "employees (employee_id, first_name, last_name, salary, department_id), departments (department_id, department_name)",
    "output": "SELECT e.first_name, e.last_name, e.salary, d.department_name FROM employees e JOIN departments d ON e.department_id = d.department_id WHERE e.salary > (SELECT AVG(salary) FROM employees WHERE department_id = e.department_id);"
  },
  {
    "instruction": "Find the number of employees hired in each month of 2021.",
    "input": "employees (employee_id, hire_date)",
    "output": "SELECT TO_CHAR(hire_date, 'YYYY-MM') AS hire_month, COUNT(employee_id) AS employee_count FROM employees WHERE EXTRACT(YEAR FROM hire_date) = 2021 GROUP BY TO_CHAR(hire_date, 'YYYY-MM') ORDER BY hire_month;"
  },
  {
    "instruction": "Find the employees with the longest tenure in each department and display their names and hire dates.",
    "input": "employees (employee_id, first_name, last_name, hire_date, department_id), departments (department_id, department_name)",
    "output": "SELECT e.first_name, e.last_name, e.hire_date, d.department_name FROM employees e JOIN departments d ON e.department_id = d.department_id WHERE e.hire_date = (SELECT MIN(hire_date) FROM employees WHERE department_id = e.department_id);"
  },
  {
    "instruction": "Find the total salary expense for employees with the job title 'IT_PROG' across all departments.",
    "input": "employees (employee_id, salary, job_id, department_id), jobs (job_id, job_title)",
    "output": "SELECT SUM(e.salary) AS total_salary FROM employees e JOIN jobs j ON e.job_id = j.job_id WHERE j.job_title = 'IT_PROG';"
  },
  {
    "instruction": "Identify employees who report to managers with a salary above the average salary of their department.",
    "input": "employees (employee_id, manager_id, salary, department_id), managers (employee_id, first_name, last_name)",
    "output": "SELECT e.first_name, e.last_name FROM employees e JOIN employees m ON e.manager_id = m.employee_id WHERE m.salary > (SELECT AVG(salary) FROM employees WHERE department_id = e.department_id);"
  },

  {
    "instruction": "List departments that have more than 5 employees earning above the average salary.",
    "input": "employees (employee_id, salary, department_id), departments (department_id, department_name)",
    "output": "SELECT d.department_name FROM departments d JOIN employees e ON d.department_id = e.department_id WHERE e.salary > (SELECT AVG(salary) FROM employees) GROUP BY d.department_name HAVING COUNT(*) > 5;"
  },
  {
    "instruction": "Find the job titles that have at least 3 employees hired in 2007.",
    "input": "employees (employee_id, hire_date, job_id), jobs (job_id, job_title)",
    "output": "SELECT j.job_title FROM jobs j JOIN employees e ON j.job_id = e.job_id WHERE EXTRACT(YEAR FROM e.hire_date) = 2007 GROUP BY j.job_title HAVING COUNT(*) >= 3;"
  },
  {
    "instruction": "Display the names of employees whose salary is higher than their manager.",
    "input": "employees (employee_id, first_name, last_name, salary, manager_id)",
    "output": "SELECT e.first_name, e.last_name FROM employees e JOIN employees m ON e.manager_id = m.employee_id WHERE e.salary > m.salary;"
  },
  {
    "instruction": "Find the departments with no employees.",
    "input": "departments (department_id, department_name), employees (employee_id, department_id)",
    "output": "SELECT d.department_name FROM departments d LEFT JOIN employees e ON d.department_id = e.department_id WHERE e.employee_id IS NULL;"
  },
  {
    "instruction": "List employees whose job title is the same as their manager’s.",
    "input": "employees (employee_id, first_name, last_name, job_id, manager_id)",
    "output": "SELECT e.first_name, e.last_name FROM employees e JOIN employees m ON e.manager_id = m.employee_id WHERE e.job_id = m.job_id;"
  },
  {
    "instruction": "Show the average salary of each job title for each department.",
    "input": "employees (employee_id, job_id, salary, department_id), jobs (job_id, job_title), departments (department_id, department_name)",
    "output": "SELECT d.department_name, j.job_title, AVG(e.salary) AS avg_salary FROM employees e JOIN jobs j ON e.job_id = j.job_id JOIN departments d ON e.department_id = d.department_id GROUP BY d.department_name, j.job_title;"
  },
  {
    "instruction": "List all regions along with the number of countries in each.",
    "input": "regions (region_id, region_name), countries (country_id, region_id)",
    "output": "SELECT r.region_name, COUNT(c.country_id) AS country_count FROM regions r LEFT JOIN countries c ON r.region_id = c.region_id GROUP BY r.region_name;"
  },
  {
    "instruction": "Find all employees who work in a country located in region 2.",
    "input": "employees (employee_id, department_id), departments (department_id, location_id), locations (location_id, country_id), countries (country_id, region_id)",
    "output": "SELECT e.employee_id FROM employees e JOIN departments d ON e.department_id = d.department_id JOIN locations l ON d.location_id = l.location_id JOIN countries c ON l.country_id = c.country_id WHERE c.region_id = 2;"
  },
  {
    "instruction": "Display the location and department names where no employees are assigned.",
    "input": "departments (department_id, department_name, location_id), locations (location_id, city), employees (employee_id, department_id)",
    "output": "SELECT l.city, d.department_name FROM departments d JOIN locations l ON d.location_id = l.location_id LEFT JOIN employees e ON d.department_id = e.department_id WHERE e.employee_id IS NULL;"
  },
  {
    "instruction": "Retrieve managers who manage more than 3 employees and have a salary above 15000.",
    "input": "employees (employee_id, manager_id, salary)",
    "output": "SELECT m.employee_id FROM employees m WHERE m.salary > 15000 AND (SELECT COUNT(*) FROM employees e WHERE e.manager_id = m.employee_id) > 3;"
  },
   {
    "instruction": "Find employees who work in a different location than their department manager.",
    "input": "employees (employee_id, first_name, last_name, department_id), departments (department_id, manager_id, location_id), locations (location_id, city)",
    "output": "SELECT e.first_name, e.last_name FROM employees e JOIN departments d ON e.department_id = d.department_id JOIN employees m ON d.manager_id = m.employee_id WHERE e.department_id IS NOT NULL AND (SELECT location_id FROM departments WHERE manager_id = m.employee_id) <> d.location_id;"
  },
  {
    "instruction": "Display the number of employees hired each year.",
    "input": "employees (employee_id, hire_date)",
    "output": "SELECT EXTRACT(YEAR FROM hire_date) AS hire_year, COUNT(*) AS total_hired FROM employees GROUP BY EXTRACT(YEAR FROM hire_date);"
  },
  {
    "instruction": "Find the highest-paid employee in each department.",
    "input": "employees (employee_id, first_name, last_name, salary, department_id)",
    "output": "SELECT first_name, last_name, salary, department_id FROM employees e WHERE salary = (SELECT MAX(salary) FROM employees WHERE department_id = e.department_id);"
  },
  {
    "instruction": "List the job titles that have no current employees assigned.",
    "input": "jobs (job_id, job_title), employees (employee_id, job_id)",
    "output": "SELECT j.job_title FROM jobs j LEFT JOIN employees e ON j.job_id = e.job_id WHERE e.job_id IS NULL;"
  },
  {
    "instruction": "Display the department with the highest total salary cost.",
    "input": "employees (salary, department_id), departments (department_id, department_name)",
    "output": "SELECT d.department_name FROM departments d JOIN employees e ON d.department_id = e.department_id GROUP BY d.department_name ORDER BY SUM(e.salary) DESC FETCH FIRST 1 ROWS ONLY;"
  },
  {
    "instruction": "List employees who were hired before their managers.",
    "input": "employees (employee_id, first_name, last_name, hire_date, manager_id)",
    "output": "SELECT e.first_name, e.last_name FROM employees e JOIN employees m ON e.manager_id = m.employee_id WHERE e.hire_date < m.hire_date;"
  },
  {
    "instruction": "Find all employees who share the same last name as at least one other employee.",
    "input": "employees (employee_id, last_name)",
    "output": "SELECT last_name FROM employees GROUP BY last_name HAVING COUNT(*) > 1;"
  },
  {
    "instruction": "Show all managers who are not assigned to any department as a department manager.",
    "input": "employees (employee_id, manager_id), departments (manager_id)",
    "output": "SELECT DISTINCT e.employee_id FROM employees e LEFT JOIN departments d ON e.employee_id = d.manager_id WHERE d.manager_id IS NULL AND EXISTS (SELECT 1 FROM employees WHERE manager_id = e.employee_id);"
  },
  {
    "instruction": "List countries with more than 2 locations.",
    "input": "locations (location_id, country_id), countries (country_id, country_name)",
    "output": "SELECT c.country_name FROM countries c JOIN locations l ON c.country_id = l.country_id GROUP BY c.country_name HAVING COUNT(l.location_id) > 2;"
  },
  {
    "instruction": "Find departments that span more than one location.",
    "input": "departments (department_id, location_id)",
    "output": "SELECT department_id FROM departments GROUP BY department_id HAVING COUNT(DISTINCT location_id) > 1;"
  },
  {
    "instruction": "Display each region and the total number of employees working in it.",
    "input": "regions (region_id, region_name), countries (country_id, region_id), locations (location_id, country_id), departments (department_id, location_id), employees (employee_id, department_id)",
    "output": "SELECT r.region_name, COUNT(e.employee_id) AS employee_count FROM regions r JOIN countries c ON r.region_id = c.region_id JOIN locations l ON c.country_id = l.country_id JOIN departments d ON l.location_id = d.location_id JOIN employees e ON d.department_id = e.department_id GROUP BY r.region_name;"
  },
  {
    "instruction": "Find the location with the highest number of departments.",
    "input": "departments (department_id, location_id), locations (location_id, city)",
    "output": "SELECT l.city FROM locations l JOIN departments d ON l.location_id = d.location_id GROUP BY l.city ORDER BY COUNT(d.department_id) DESC FETCH FIRST 1 ROWS ONLY;"
  },
  {
    "instruction": "Find the top 3 employees with the longest tenure.",
    "input": "employees (employee_id, first_name, last_name, hire_date)",
    "output": "SELECT first_name, last_name, hire_date FROM employees ORDER BY hire_date ASC FETCH FIRST 3 ROWS ONLY;"
  },
  {
    "instruction": "List employees along with the region they work in.",
    "input": "employees (employee_id, department_id), departments (department_id, location_id), locations (location_id, country_id), countries (country_id, region_id), regions (region_id, region_name)",
    "output": "SELECT e.employee_id, r.region_name FROM employees e JOIN departments d ON e.department_id = d.department_id JOIN locations l ON d.location_id = l.location_id JOIN countries c ON l.country_id = c.country_id JOIN regions r ON c.region_id = r.region_id;"
  },
  {
    "instruction": "Display the percentage of employees per job title.",
    "input": "employees (employee_id, job_id), jobs (job_id, job_title)",
    "output": "SELECT j.job_title, ROUND((COUNT(e.employee_id) * 100.0) / (SELECT COUNT(*) FROM employees), 2) AS percentage FROM jobs j LEFT JOIN employees e ON j.job_id = e.job_id GROUP BY j.job_title;"
  },
  {
    "instruction": "Identify departments where the average salary is below 5000.",
    "input": "employees (employee_id, salary, department_id), departments (department_id, department_name)",
    "output": "SELECT d.department_name FROM employees e JOIN departments d ON e.department_id = d.department_id GROUP BY d.department_name HAVING AVG(e.salary) < 5000;"
  },
  {
    "instruction": "List employees with the same salary and job title as someone in department 90.",
    "input": "employees (employee_id, first_name, salary, job_id, department_id)",
    "output": "SELECT DISTINCT e1.first_name FROM employees e1 JOIN employees e2 ON e1.salary = e2.salary AND e1.job_id = e2.job_id WHERE e2.department_id = 90 AND e1.employee_id <> e2.employee_id;"
  },
  {
    "instruction": "Show departments where all employees earn above 3000.",
    "input": "employees (employee_id, salary, department_id), departments (department_id, department_name)",
    "output": "SELECT d.department_name FROM departments d JOIN employees e ON d.department_id = e.department_id GROUP BY d.department_name HAVING MIN(e.salary) > 3000;"
  },
  {
    "instruction": "Find the department(s) with the most employees.",
    "input": "employees (employee_id, department_id), departments (department_id, department_name)",
    "output": "SELECT d.department_name FROM departments d JOIN employees e ON d.department_id = e.department_id GROUP BY d.department_name HAVING COUNT(e.employee_id) = (SELECT MAX(emp_count) FROM (SELECT COUNT(*) AS emp_count FROM employees GROUP BY department_id));"
  },
  {
    "instruction": "Display the count of employees by city.",
    "input": "employees (employee_id, department_id), departments (department_id, location_id), locations (location_id, city)",
    "output": "SELECT l.city, COUNT(e.employee_id) AS employee_count FROM employees e JOIN departments d ON e.department_id = d.department_id JOIN locations l ON d.location_id = l.location_id GROUP BY l.city;"
  },
  {
    "instruction": "List regions that have more than one country associated with them.",
    "input": "regions (region_id, region_name), countries (country_id, region_id)",
    "output": "SELECT r.region_name FROM regions r JOIN countries c ON r.region_id = c.region_id GROUP BY r.region_name HAVING COUNT(DISTINCT c.country_id) > 1;"
  },
  {
    "instruction": "Display cities where the number of departments is more than two.",
    "input": "locations (location_id, city), departments (department_id, location_id)",
    "output": "SELECT l.city FROM locations l JOIN departments d ON l.location_id = d.location_id GROUP BY l.city HAVING COUNT(d.department_id) > 2;"
  },
  {
    "instruction": "Find employees whose job title is not 'IT_PROG' but are in the same department as an 'IT_PROG'.",
    "input": "employees (employee_id, first_name, department_id, job_id), jobs (job_id, job_title)",
    "output": "SELECT DISTINCT e1.first_name FROM employees e1 JOIN employees e2 ON e1.department_id = e2.department_id WHERE e2.job_id = 'IT_PROG' AND e1.job_id <> 'IT_PROG';"
  },
  {
    "instruction": "Identify countries that have no employees assigned to them through departments and locations.",
    "input": "countries (country_id, country_name), locations (location_id, country_id), departments (department_id, location_id), employees (employee_id, department_id)",
    "output": "SELECT DISTINCT c.country_name FROM countries c LEFT JOIN locations l ON c.country_id = l.country_id LEFT JOIN departments d ON l.location_id = d.location_id LEFT JOIN employees e ON d.department_id = e.department_id WHERE e.employee_id IS NULL;"
  },
  {
    "instruction": "Show job titles with the highest average salary.",
    "input": "employees (employee_id, salary, job_id), jobs (job_id, job_title)",
    "output": "SELECT j.job_title FROM jobs j JOIN employees e ON j.job_id = e.job_id GROUP BY j.job_title ORDER BY AVG(e.salary) DESC FETCH FIRST 1 ROWS ONLY;"
  },
  {
    "instruction": "Find all departments where the manager does not belong to that department.",
    "input": "departments (department_id, manager_id), employees (employee_id, department_id)",
    "output": "SELECT d.department_id FROM departments d JOIN employees e ON d.manager_id = e.employee_id WHERE d.department_id <> e.department_id;"
  },
  {
    "instruction": "List locations that do not have any departments.",
    "input": "locations (location_id, city), departments (department_id, location_id)",
    "output": "SELECT l.city FROM locations l LEFT JOIN departments d ON l.location_id = d.location_id WHERE d.department_id IS NULL;"
  },
  {
    "instruction": "Find all countries that belong to regions that have less than 2 countries.",
    "input": "countries (country_id, region_id), regions (region_id, region_name)",
    "output": "SELECT c.country_id FROM countries c WHERE c.region_id IN (SELECT region_id FROM countries GROUP BY region_id HAVING COUNT(*) < 2);"
  },
  {
    "instruction": "Display job titles held by employees in departments located in the United States.",
    "input": "employees (employee_id, job_id, department_id), jobs (job_id, job_title), departments (department_id, location_id), locations (location_id, country_id), countries (country_id, country_name)",
    "output": "SELECT DISTINCT j.job_title FROM employees e JOIN jobs j ON e.job_id = j.job_id JOIN departments d ON e.department_id = d.department_id JOIN locations l ON d.location_id = l.location_id JOIN countries c ON l.country_id = c.country_id WHERE c.country_name = 'United States of America';"
  },
  {
    "instruction": "List departments where all employees earn more than their manager.",
    "input": "employees (employee_id, salary, department_id, manager_id)",
    "output": "SELECT e.department_id FROM employees e JOIN employees m ON e.manager_id = m.employee_id GROUP BY e.department_id, m.salary HAVING MIN(e.salary) > MAX(m.salary);"
  },
  {
    "instruction": "Show the job titles of managers who manage more than 3 departments.",
    "input": "departments (department_id, manager_id), employees (employee_id, job_id), jobs (job_id, job_title)",
    "output": "SELECT j.job_title FROM departments d JOIN employees e ON d.manager_id = e.employee_id JOIN jobs j ON e.job_id = j.job_id GROUP BY j.job_title, e.employee_id HAVING COUNT(d.department_id) > 3;"
  },
  {
    "instruction": "Find the region with the most employees.",
    "input": "employees (employee_id, department_id), departments (department_id, location_id), locations (location_id, country_id), countries (country_id, region_id), regions (region_id, region_name)",
    "output": "SELECT r.region_name FROM employees e JOIN departments d ON e.department_id = d.department_id JOIN locations l ON d.location_id = l.location_id JOIN countries c ON l.country_id = c.country_id JOIN regions r ON c.region_id = r.region_id GROUP BY r.region_name ORDER BY COUNT(e.employee_id) DESC FETCH FIRST 1 ROWS ONLY;"
  },
  {
    "instruction": "List departments that have employees with multiple job titles.",
    "input": "employees (employee_id, job_id, department_id)",
    "output": "SELECT department_id FROM employees GROUP BY department_id HAVING COUNT(DISTINCT job_id) > 1;"
  },
  {
    "instruction": "Find countries where no departments exist.",
    "input": "countries (country_id, country_name), locations (location_id, country_id), departments (department_id, location_id)",
    "output": "SELECT DISTINCT c.country_name FROM countries c LEFT JOIN locations l ON c.country_id = l.country_id LEFT JOIN departments d ON l.location_id = d.location_id WHERE d.department_id IS NULL;"
  },
  {
    "instruction": "Show all employees working in the same city as their department manager.",
    "input": "employees (employee_id, first_name, department_id), departments (department_id, manager_id, location_id), locations (location_id, city)",
    "output": "SELECT e.first_name FROM employees e JOIN departments d ON e.department_id = d.department_id JOIN employees m ON d.manager_id = m.employee_id JOIN departments dm ON m.department_id = dm.department_id JOIN locations l1 ON d.location_id = l1.location_id JOIN locations l2 ON dm.location_id = l2.location_id WHERE l1.city = l2.city;"
  },
  {
    "instruction": "List the number of employees who share both job title and salary.",
    "input": "employees (employee_id, job_id, salary)",
    "output": "SELECT job_id, salary, COUNT(*) AS count FROM employees GROUP BY job_id, salary HAVING COUNT(*) > 1;"
  },
  {
    "instruction": "Find the department and city with the lowest average salary.",
    "input": "employees (salary, department_id), departments (department_id, location_id), locations (location_id, city)",
    "output": "SELECT d.department_id, l.city FROM employees e JOIN departments d ON e.department_id = d.department_id JOIN locations l ON d.location_id = l.location_id GROUP BY d.department_id, l.city ORDER BY AVG(e.salary) ASC FETCH FIRST 1 ROWS ONLY;"
  },
  {
    "instruction": "Find managers who manage departments in different countries.",
    "input": "departments (department_id, manager_id, location_id), locations (location_id, country_id)",
    "output": "SELECT manager_id FROM departments d JOIN locations l ON d.location_id = l.location_id GROUP BY manager_id HAVING COUNT(DISTINCT l.country_id) > 1;"
  },
  {
    "instruction": "List all employees and their department's country name.",
    "input": "employees (employee_id, department_id), departments (department_id, location_id), locations (location_id, country_id), countries (country_id, country_name)",
    "output": "SELECT e.employee_id, c.country_name FROM employees e JOIN departments d ON e.department_id = d.department_id JOIN locations l ON d.location_id = l.location_id JOIN countries c ON l.country_id = c.country_id;"
  },
  {
    "instruction": "List departments that only have employees with the same job title.",
    "input": "employees (employee_id, department_id, job_id)",
    "output": "SELECT department_id FROM employees GROUP BY department_id HAVING COUNT(DISTINCT job_id) = 1;"
  },
  {
    "instruction": "Find job titles of employees hired in the same month as their manager.",
    "input": "employees (employee_id, job_id, hire_date, manager_id), jobs (job_id, job_title)",
    "output": "SELECT DISTINCT j.job_title FROM employees e JOIN employees m ON e.manager_id = m.employee_id JOIN jobs j ON e.job_id = j.job_id WHERE TO_CHAR(e.hire_date, 'MM') = TO_CHAR(m.hire_date, 'MM');"
  },
  {
    "instruction": "List the department names where no employees are currently working.",
    "input": "departments (department_id, department_name), employees (employee_id, department_id)",
    "output": "SELECT d.department_name FROM departments d LEFT JOIN employees e ON d.department_id = e.department_id WHERE e.employee_id IS NULL;"
  },
  {
    "instruction": "List job titles where the maximum salary is below 5000.",
    "input": "employees (employee_id, salary, job_id), jobs (job_id, job_title)",
    "output": "SELECT j.job_title FROM jobs j JOIN employees e ON j.job_id = e.job_id GROUP BY j.job_title HAVING MAX(e.salary) < 5000;"
  },
  {
    "instruction": "Display the average salary per region.",
    "input": "employees (employee_id, salary, department_id), departments (department_id, location_id), locations (location_id, country_id), countries (country_id, region_id), regions (region_id, region_name)",
    "output": "SELECT r.region_name, ROUND(AVG(e.salary), 2) AS avg_salary FROM employees e JOIN departments d ON e.department_id = d.department_id JOIN locations l ON d.location_id = l.location_id JOIN countries c ON l.country_id = c.country_id JOIN regions r ON c.region_id = r.region_id GROUP BY r.region_name;"
  }

]

# Convert to HuggingFace dataset
dataset = Dataset.from_list(data)

alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # do not forget this part!
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN # without this token generation goes on forever!
        texts.append(text)
    return { "text" : texts, }

# Apply formatting
dataset = dataset.map(formatting_prompts_func, batched=True)


Map:   0%|          | 0/107 [00:00<?, ? examples/s]

In [42]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = 4096,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 100, # increase this to make the model learn "better"
        num_train_epochs=4,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/107 [00:00<?, ? examples/s]

In [43]:
# We're now kicking off the actual training of our model, which will spit out some statistics showing us how well it learns
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 107 | Num Epochs = 8 | Total steps = 100
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 39,976,960/7,000,000,000 (0.57% trained)


Step,Training Loss
1,0.957600
2,1.035000
3,0.907300
4,0.719400
5,0.523800
6,0.424500
7,0.306700
8,0.300000
9,0.337000
10,0.340400


In [44]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Display employees who do not have commission", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 128, use_cache = True)
tokenizer.batch_decode(outputs)

['<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nDisplay employees who do not have commission\n\n### Input:\n\n\n### Response:\nSELECT first_name FROM employees WHERE commission_pct IS NULL;</s>']